In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent root directory and add to sys.path
import sys, os
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
from src.rocket import Rocket
from src.pos_rocket_vis import *
from LandMPC_template.nmpc_land import NmpcCtrl
import numpy as np

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir,"rocket.yaml")

# Rocket setup
Ts  = 1/20
rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
rocket.mass = 1.7 # Do not change!!!

# Visualization setup
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1

In [ ]:
sim_time = 10.0 # simulation length in seconds
# x0 = [w(3), r(3), v(3), p(3)]
# r = [alpha, beta, gamma] (roll, pitch, yaw)
# p = [x, y, z]
# Start: x=3, y=2, z=10, roll=30 deg (gamma=30 deg)
x0 = np.array([0,0,0, 0,0,np.deg2rad(30), 0,0,0, 3,2,10]) 
# Target: x=1, y=0, z=3, roll=0 deg
x_ref = np.array([0,0,0, 0,0,0, 0,0,0, 1,0,3])
xs, us = rocket.trim(x_ref)
print("Linearization around a steady state:")
print("x_ref = ", x_ref)
print("xs = ", xs)
print("us = ", us)

H = 2.0 # horizon time 
nmpc = NmpcCtrl(rocket, H, xs, us)

In [ ]:
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol = rocket.simulate_land(nmpc, sim_time, H, x0)

In [ ]:
vis.animate(t_cl[:-1], x_cl[:,:-1], u_cl, T_ol=t_ol[...,:-1], X_ol=x_ol, U_ol=u_ol)
plot_static_states_inputs(t_cl[:-1], x_cl[:,:-1], u_cl, xs)